In [1]:
from comet_ml import Experiment
import os
import torch

from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from concrete_cracks_dataset import ConcreteCracksDataset
from concrete_cracks_model import ConcreteCracksModel
import concrete_cracks_utils as cc_utils
from Unet import Unet

API_KEY = os.environ['COMETKEY']
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Important note, if batch requires more memory than gpu has it starts to copy values, because of that performance drastically decrease. It is about 25 times slower
hyper_params = {
    'learning_rate': 0.0003,
    'epochs': 10,
    'batch_size': 5, 
    'n_negative': 3,
    'n_positive': 7,
}

C:\Users\karly\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
experiment = Experiment(
  api_key = API_KEY,
  project_name = 'concrete-cracks-segmentation',
  workspace='my-projects'
)
experiment.log_parameters(hyper_params)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/my-projects/concrete-cracks-segmentation/7dfb0c7f827e4b49bac7a30117527f3c



In [3]:
model = Unet(ch_in=3, ch_out=1).to(DEVICE)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyper_params['learning_rate'])
scaler = torch.cuda.amp.grad_scaler.GradScaler()


train_dataset = ConcreteCracksDataset(hyper_params['n_negative'], hyper_params['n_positive'], skip=10, transform=ToTensor())
train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=5,
    shuffle=True,
    num_workers=1
)

val_dataset = ConcreteCracksDataset(0, 5, skip=0, transform=ToTensor())
val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1
)

In [4]:
for epoch in range(hyper_params['epochs']):
    cc_utils.train_fn(train_loader, model, DEVICE, optimizer, loss_fn, scaler)
    cc_utils.log_images(val_loader, model, DEVICE, epoch, experiment)
    f1_score = cc_utils.check_accuracy(val_loader, model, DEVICE)
    experiment.log_metric('f1_score', f1_score, epoch=epoch)

100%|██████████| 2/2 [00:04<00:00,  2.07s/it, loss=0.588]


Showing results

In [ ]:
import matplotlib.pyplot as plt
image, mask = next(iter(val_loader))

image = image.to(DEVICE)
mask = mask.to(DEVICE)

# it is important to use it like to to decrease memory usage
model.eval()
with torch.no_grad():
    predict = model(image)
model.train()

In [ ]:
i = 8 # i in range(0, batch_size-1)
img_dim = image[i].permute(1, 2, 0).cpu()
predict_dim = predict[i].permute(1,2,0).cpu()
msk_dim = mask[i].permute(1,2,0).cpu()
plt.imshow(img_dim)
plt.show()
plt.imshow(predict_dim)
plt.show()
plt.imshow(msk_dim*255)

In [ ]:
model_state_file = 'model_versions/filename'
torch.save(model.state_dict(), model_state_file)
#experiment.log_model('name on comet', model_state_file)

In [ ]:
experiment.end()